[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/09-route-filter.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/00-introduction.ipynb)

# Semantic Router Filter

The Semantic Router library can be used as a super fast route making layer on top of LLMs. That means rather than waiting on a slow agent to decide what to do, we can use the magic of semantic vector space to make routes. Cutting route making time down from seconds to milliseconds.

In [3]:
!pip install "pinecone[grpc]"

  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl.metadata (1.5 kB)
Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl (221 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.3 MB 8.4 MB/s eta 0:00:01
   ------------ --------------------------- 1.3/4.3 MB 6.1 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.3 MB 4.6 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.3 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------  4.2/4.3 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 4.2 MB/s eta 0:00:00
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl (7.9 kB)


In [18]:
from pinecone.grpc import PineconeGRPC, GRPCClientConfig
from pinecone import ServerlessSpec
import time

# Initialize a client.
# API key is required, but the value does not matter.
# Host and port of the Pinecone Local instance
# is required when starting without indexes.
pc = PineconeGRPC(api_key="pclocal", host="http://localhost:5080")

# Create an index
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )

# Wait for the index to be ready
while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

# Target the index, disabling tls
index_host = pc.describe_index(index_name).host
index = pc.Index(host=index_host, grpc_config=GRPCClientConfig(secure=False))

# Upsert records into the index
index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5], "metadata": {"genre": "drama"}},
        {"id": "vec2", "values": [3.0, -2.0], "metadata": {"genre": "documentary"}},
        {"id": "vec3", "values": [0.5, -1.5], "metadata": {"genre": "documentary"}},
    ],
    namespace="example-namespace",
)

# Check the number of records in the index
print("Index stats:\n", index.describe_index_stats())

# Query the index with a metadata filter
response = index.query(
    vector=[3.0, -2.0],
    filter={"genre": {"$eq": "documentary"}},
    top_k=1,
    include_values=True,
    include_metadata=True,
    namespace="example-namespace",
)

print("\nQuery response:\n", response)

# Delete the index
pc.delete_index(index_name)

Index stats:
 {'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 3}},
 'total_vector_count': 3}

Query response:
 {'matches': [{'id': 'vec2',
              'metadata': {'genre': 'documentary'},
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [3.0, -2.0]}],
 'namespace': 'example-namespace'}


## Getting Started

We start by installing the library:

In [ ]:
!pip install -qU semantic-router

We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [ ]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's define another for good measure:

In [ ]:
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

Now we initialize our embedding model:

In [ ]:
import os
from getpass import getpass
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
    "Enter Cohere API Key: "
)
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
#     "Enter OpenAI API Key: "
# )

encoder = CohereEncoder()
# encoder = OpenAIEncoder()

Now we define the `RouteLayer`. When called, the route layer will consume text (a query) and output the category (`Route`) it belongs to — to initialize a `RouteLayer` we need our `encoder` model and a list of `routes`.

In [ ]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes)

2024-05-07 16:02:43 INFO semantic_router.utils.logger local


Now we can test it:

In [ ]:
rl("don't you love politics?")

RouteChoice(name='politics', function_call=None, similarity_score=None)

In [ ]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None, similarity_score=None)

Both are classified accurately, what if we send a query that is unrelated to our existing `Route` objects?

In [ ]:
rl("I'm interested in learning about llama 2")

RouteChoice(name=None, function_call=None, similarity_score=None)

In this case, we return `None` because no matches were identified.

# Demonstrating the Filter Feature

Now, let's demonstrate the filter feature. We can specify a subset of routes to consider when making a classification. This can be useful if we want to restrict the scope of possible routes based on some context.

For example, let's say we only want to consider the "chitchat" route for a particular query:

In [ ]:
rl("don't you love politics?", route_filter=["chitchat"])

RouteChoice(name='chitchat', function_call=None, similarity_score=None)

Even though the query might be more related to the "politics" route, it will be classified as "chitchat" because we've restricted the routes to consider.

Similarly, we can restrict it to the "politics" route:

In [ ]:
rl("how's the weather today?", route_filter=["politics"])

RouteChoice(name=None, function_call=None, similarity_score=None)

In this case, it will return None because the query doesn't match the "politics" route well enough to pass the threshold.

